In [1]:
#imports
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor

In [2]:
t_data = pd.read_csv('train.csv', index_col = 'PassengerId')
t_test = pd.read_csv('test.csv', index_col = 'PassengerId')
t_data

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [3]:
### 

### Check out column info

In [4]:
t_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


### Drop Cabin (77% Null) , Ticket (unique objects), and Name (unique objects)

In [5]:
t_data.drop(["Cabin","Ticket", "Name"], axis='columns', inplace = True)
t_test.drop(["Cabin","Ticket", "Name"], axis='columns', inplace = True)
t_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,S
2,1,1,female,38.0,1,0,71.2833,C
3,1,3,female,26.0,0,0,7.9250,S
4,1,1,female,35.0,1,0,53.1000,S
5,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
887,0,2,male,27.0,0,0,13.0000,S
888,1,1,female,19.0,0,0,30.0000,S
889,0,3,female,NaN,1,2,23.4500,S


### Change Sex to Boolean

In [6]:
t_data.loc[t_data.Sex == 'male', 'Sex'] = False
t_data.loc[t_data.Sex == 'female', 'Sex'] = True
t_data.rename(columns = {'Sex':'IsFemale'}, inplace = True)
#t_test
t_test.loc[t_test.Sex == 'male', 'Sex'] = False
t_test.loc[t_test.Sex == 'female', 'Sex'] = True
t_test.rename(columns = {'Sex':'IsFemale'}, inplace = True)

t_data


,Survived,Pclass,IsFemale,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,False,22.0,1,0,7.2500,S
2,1,1,True,38.0,1,0,71.2833,C
3,1,3,True,26.0,0,0,7.9250,S
4,1,1,True,35.0,1,0,53.1000,S
5,0,3,False,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
887,0,2,False,27.0,0,0,13.0000,S
888,1,1,True,19.0,0,0,30.0000,S
889,0,3,True,NaN,1,2,23.4500,S


### Drop all NaN -- fix this in next version.  Age is 20% NaN so dont drop that.  Decide whether to drop the 3 rows with embarked NaN

In [7]:
t_data = t_data.fillna(0)

t_test = t_test.fillna(0)

In [8]:
t_data

,Survived,Pclass,IsFemale,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,False,22.0,1,0,7.2500,S
2,1,1,True,38.0,1,0,71.2833,C
3,1,3,True,26.0,0,0,7.9250,S
4,1,1,True,35.0,1,0,53.1000,S
5,0,3,False,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
887,0,2,False,27.0,0,0,13.0000,S
888,1,1,True,19.0,0,0,30.0000,S
889,0,3,True,0.0,1,2,23.4500,S


### One-Hot Encoding for Embarked (Next Version)

In [9]:
# # Apply one-hot encoder to Embarked
# OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
# t_data(OH_encoder.fit_transform(t_data['Embarked']), inplace = True)
# # One-hot encoding removed index; put it back
# # t_data.index = X_train.index
# # OH_cols_valid.index = X_valid.index

# # Remove categorical columns (will replace with one-hot encoding)
# t_data.drop('Embarked', axis=1)
# # num_X_valid = X_valid.drop(object_cols, axis=1)

# t_data

### Cross Validation

In [10]:
features = ['Pclass','IsFemale','Age','SibSp','Parch','Fare']
X = t_data[features]
y = t_data.Survived
model = XGBRegressor(n_estimators=500)
cv = cross_val_score(model, X, y, cv = 5)
cv.mean()

0.26805303256895263

### XGBoost

In [11]:
model.fit(X,y)
preds = model.predict(t_test[features])
preds = preds.astype(int)

In [15]:
output = pd.DataFrame({'PassengerId': t_test.index,
                       'Survived': preds})
output.to_csv('submission2.csv', index=False)

In [16]:
sub = pd.read_csv('submission2.csv')
sub
sub.Survived.value_counts()

0    363
1     55
Name: Survived, dtype: int64